# 実験用DuckDBの作成

## 目的

4つのWikipediaデータセット（英語/日本語 × タイトル/本文）を統合し、
E5-largeでembeddingを生成してDuckDBに保存する。

## データ

| ファイル | 件数 | 内容 |
|----------|------|------|
| wikipedia_titles_en_100k | 100,000 | 英語タイトル |
| wikipedia_titles_ja_100k | 100,000 | 日本語タイトル |
| wikipedia_en_body_100k | 100,000 | 英語本文 |
| wikipedia_ja_body_100k | 100,000 | 日本語本文 |

In [2]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import duckdb

print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'CUDA device: {torch.cuda.get_device_name(0)}')

PyTorch: 2.10.0+cu128
CUDA available: True
CUDA device: NVIDIA GeForce RTX 4090


## 1. データの読み込みと統合

In [3]:
DATA_DIR = '../data'

# 4つのデータを読み込み
print('データ読み込み中...')

# タイトルのみ
titles_en = pd.read_parquet(f'{DATA_DIR}/wikipedia_titles_en_100k.parquet')
titles_ja = pd.read_parquet(f'{DATA_DIR}/wikipedia_titles_ja_100k.parquet')

# 本文あり
body_en = pd.read_parquet(f'{DATA_DIR}/wikipedia_en_body_100k.parquet')
body_ja = pd.read_parquet(f'{DATA_DIR}/wikipedia_ja_body_100k.parquet')

print(f'英語タイトル: {len(titles_en):,}件')
print(f'日本語タイトル: {len(titles_ja):,}件')
print(f'英語本文: {len(body_en):,}件')
print(f'日本語本文: {len(body_ja):,}件')

データ読み込み中...
英語タイトル: 100,000件
日本語タイトル: 100,000件
英語本文: 100,000件
日本語本文: 100,000件


In [4]:
# 統合用のデータフレームを作成
def prepare_dataset(df, dataset_name, has_body=False):
    """データセットを統一フォーマットに変換"""
    result = pd.DataFrame()
    result['original_id'] = df['id']
    result['title'] = df['title']
    result['lang'] = df['lang']
    result['dataset'] = dataset_name
    
    if has_body and 'text' in df.columns:
        # タイトル + 本文を結合
        result['text'] = df['title'] + '\n\n' + df['text'].fillna('')
    else:
        # タイトルのみ
        result['text'] = df['title']
    
    return result

# 各データセットを変換
datasets = [
    prepare_dataset(titles_en, 'titles_en', has_body=False),
    prepare_dataset(titles_ja, 'titles_ja', has_body=False),
    prepare_dataset(body_en, 'body_en', has_body=True),
    prepare_dataset(body_ja, 'body_ja', has_body=True),
]

# 統合
all_data = pd.concat(datasets, ignore_index=True)
all_data['id'] = range(len(all_data))  # 連番ID

print(f'\n統合データ: {len(all_data):,}件')
print(f'\nデータセット別件数:')
print(all_data['dataset'].value_counts())


統合データ: 400,000件

データセット別件数:
dataset
titles_en    100000
titles_ja    100000
body_en      100000
body_ja      100000
Name: count, dtype: int64


In [5]:
# テキスト長の確認
all_data['text_len'] = all_data['text'].str.len()

print('テキスト長の統計:')
print(all_data.groupby('dataset')['text_len'].describe().round(0))

テキスト長の統計:
              count    mean      std   min    25%     50%     75%        max
dataset                                                                     
body_en    100000.0  7216.0  26580.0  18.0  615.0  2752.0  6642.0  2084288.0
body_ja    100000.0  4736.0  10274.0  17.0  937.0  2233.0  4733.0   693922.0
titles_en  100000.0    28.0     16.0   1.0   15.0    24.0    38.0      238.0
titles_ja  100000.0    12.0      9.0   1.0    5.0    10.0    16.0      124.0


In [6]:
# テキストを切り詰め（E5-largeの制限対応）
# E5-largeは512トークン制限だが、文字数で約2000文字程度を目安に
MAX_CHARS = 2000

all_data['text_truncated'] = all_data['text'].str[:MAX_CHARS]

# 切り詰め後の長さ確認
all_data['truncated_len'] = all_data['text_truncated'].str.len()
print(f'切り詰め後の最大長: {all_data["truncated_len"].max()}')
print(f'切り詰められたレコード数: {(all_data["text_len"] > MAX_CHARS).sum():,}件')

切り詰め後の最大長: 2000
切り詰められたレコード数: 111,419件


## 2. E5-largeでEmbedding生成

In [7]:
# E5-largeモデルの読み込み
print('E5-largeモデルを読み込み中...')
model = SentenceTransformer('intfloat/multilingual-e5-large', device='cuda')
print(f'モデル読み込み完了')
print(f'Embedding次元: {model.get_sentence_embedding_dimension()}')

E5-largeモデルを読み込み中...
モデル読み込み完了
Embedding次元: 1024


In [8]:
# バッチでembedding生成
BATCH_SIZE = 64

texts = all_data['text_truncated'].tolist()
# E5用のプレフィックス追加
texts_with_prefix = ['passage: ' + t for t in texts]

print(f'Embedding生成開始: {len(texts):,}件')
print(f'バッチサイズ: {BATCH_SIZE}')

embeddings = []
for i in tqdm(range(0, len(texts_with_prefix), BATCH_SIZE)):
    batch = texts_with_prefix[i:i+BATCH_SIZE]
    batch_embeddings = model.encode(batch, convert_to_numpy=True, show_progress_bar=False)
    embeddings.append(batch_embeddings)

embeddings = np.vstack(embeddings)
print(f'\nEmbedding形状: {embeddings.shape}')

Embedding生成開始: 400,000件
バッチサイズ: 64


100%|██████████| 6250/6250 [34:09<00:00,  3.05it/s]



Embedding形状: (400000, 1024)


In [9]:
# embeddingをデータフレームに追加
all_data['embedding'] = list(embeddings)

# 不要なカラムを削除
all_data = all_data.drop(columns=['text_len', 'truncated_len'])

print(f'最終カラム: {list(all_data.columns)}')
print(f'データ数: {len(all_data):,}')

最終カラム: ['original_id', 'title', 'lang', 'dataset', 'text', 'id', 'text_truncated', 'embedding']
データ数: 400,000


## 3. DuckDBに保存（HNSWインデックス付き）

In [ ]:
# DuckDBファイルを作成
DB_PATH = '../data/experiment_400k.duckdb'

# 既存ファイルがあれば削除
import os
if os.path.exists(DB_PATH):
    os.remove(DB_PATH)
    print(f'既存ファイルを削除: {DB_PATH}')

# 接続
con = duckdb.connect(DB_PATH)

# VSS拡張をインストール・ロード
con.execute("INSTALL vss")
con.execute("LOAD vss")

# HNSWインデックスの永続化を有効化
con.execute("SET hnsw_enable_experimental_persistence = true")

print('DuckDB接続完了')
print('VSS拡張ロード完了')
print('HNSW永続化有効化完了')

In [11]:
# テーブル作成
con.execute("""
    CREATE TABLE documents (
        id INTEGER PRIMARY KEY,
        original_id INTEGER,
        title VARCHAR,
        lang VARCHAR,
        dataset VARCHAR,
        text VARCHAR,
        text_truncated VARCHAR,
        embedding FLOAT[1024]
    )
""")

print('テーブル作成完了')

テーブル作成完了


In [12]:
# データ挿入（バッチ処理）
INSERT_BATCH = 10000

print(f'データ挿入中: {len(all_data):,}件')

for i in tqdm(range(0, len(all_data), INSERT_BATCH)):
    batch = all_data.iloc[i:i+INSERT_BATCH]
    
    # embedding をリストに変換
    records = []
    for _, row in batch.iterrows():
        records.append((
            int(row['id']),
            int(row['original_id']),
            row['title'],
            row['lang'],
            row['dataset'],
            row['text'],
            row['text_truncated'],
            row['embedding'].tolist()
        ))
    
    con.executemany("""
        INSERT INTO documents VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, records)

print('データ挿入完了')

データ挿入中: 400,000件


100%|██████████| 40/40 [30:43<00:00, 46.08s/it]

データ挿入完了


In [14]:
# HNSW永続化を有効化
con.execute("SET hnsw_enable_experimental_persistence = true")
print('HNSW永続化有効化完了')

HNSW永続化有効化完了


In [15]:
# HNSWインデックス作成
print('HNSWインデックス作成中...')

con.execute("""
    CREATE INDEX hnsw_idx ON documents 
    USING HNSW (embedding) 
    WITH (metric = 'cosine')
""")

print('HNSWインデックス作成完了')

HNSWインデックス作成中...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

HNSWインデックス作成完了


In [16]:
# 確認
print('=' * 80)
print('データベース確認')
print('=' * 80)

count = con.execute("SELECT COUNT(*) FROM documents").fetchone()[0]
print(f'総レコード数: {count:,}')

print('\nデータセット別件数:')
result = con.execute("""
    SELECT dataset, lang, COUNT(*) as cnt 
    FROM documents 
    GROUP BY dataset, lang
    ORDER BY dataset
""").fetchdf()
print(result.to_string(index=False))

# サンプルデータ
print('\nサンプルデータ:')
sample = con.execute("""
    SELECT id, title, lang, dataset, LENGTH(text_truncated) as text_len
    FROM documents 
    LIMIT 5
""").fetchdf()
print(sample.to_string(index=False))

データベース確認
総レコード数: 400,000

データセット別件数:
  dataset lang    cnt
  body_en   en 100000
  body_ja   ja 100000
titles_en   en 100000
titles_ja   ja 100000

サンプルデータ:
 id                                                              title lang   dataset  text_len
  0                Category:People by populated place in County Galway   en titles_en        51
  1                                                            Sulkowo   en titles_en         7
  2                                                          Felleries   en titles_en         9
  3 Wikipedia:WikiProject Spam/LinkReports/comprarviagragenericoes.net   en titles_en        66
  4                        Wikipedia:Articles for deletion/Hamdy Ahmed   en titles_en        43


In [17]:
# 類似検索テスト
print('\n' + '=' * 80)
print('類似検索テスト')
print('=' * 80)

# テストクエリ
test_query = "機械学習とは何ですか"
query_embedding = model.encode(['query: ' + test_query], convert_to_numpy=True)[0]

print(f'クエリ: {test_query}')
print('\nTop-5 類似結果:')

result = con.execute("""
    SELECT id, title, lang, dataset,
           array_cosine_similarity(embedding, ?::FLOAT[1024]) as similarity
    FROM documents
    ORDER BY similarity DESC
    LIMIT 5
""", [query_embedding.tolist()]).fetchdf()

print(result.to_string(index=False))


類似検索テスト
クエリ: 機械学習とは何ですか

Top-5 類似結果:
    id                  title lang   dataset  similarity
303584  Transformer (機械学習モデル)   ja   body_ja    0.821834
365449 Wikipedia:削除依頼/ニューラル機械   ja   body_ja    0.817743
365320                   機械翻訳   ja   body_ja    0.817332
341690               アンドリュー・ン   ja   body_ja    0.816761
110973              ディープラーニング   ja titles_ja    0.816399


In [18]:
# クローズ
con.close()

# ファイルサイズ確認
size_mb = os.path.getsize(DB_PATH) / (1024 * 1024)
print(f'\nDuckDBファイル: {DB_PATH}')
print(f'ファイルサイズ: {size_mb:.1f} MB')


DuckDBファイル: ../data/experiment_400k.duckdb
ファイルサイズ: 6844.8 MB


## 4. 完了サマリー

<cell_type>markdown</cell_type>## 5. データベース検証

In [20]:
# データベース検証
import os

# 再接続（読み取り専用）
con = duckdb.connect(DB_PATH, read_only=True)
con.execute("LOAD vss")

print('=' * 80)
print('実験用DuckDB 検証結果')
print('=' * 80)

# ファイルサイズ
size_mb = os.path.getsize(DB_PATH) / (1024 * 1024)
print(f'\nファイル: {DB_PATH}')
print(f'サイズ: {size_mb:.1f} MB')

# レコード数
count = con.execute("SELECT COUNT(*) FROM documents").fetchone()[0]
print(f'\n総レコード数: {count:,}')

# データセット別件数
print('\nデータセット別件数:')
result = con.execute("""
    SELECT dataset, lang, COUNT(*) as cnt 
    FROM documents 
    GROUP BY dataset, lang
    ORDER BY dataset
""").fetchdf()
print(result.to_string(index=False))

# インデックス確認
print('\nインデックス:')
indexes = con.execute("SELECT index_name, table_name, is_unique FROM duckdb_indexes()").fetchdf()
print(indexes.to_string(index=False))

# Embedding次元確認
emb_dim = con.execute("SELECT len(embedding) FROM documents LIMIT 1").fetchone()[0]
print(f'\nEmbedding次元: {emb_dim}')

実験用DuckDB 検証結果

ファイル: ../data/experiment_400k.duckdb
サイズ: 6844.8 MB

総レコード数: 400,000

データセット別件数:
  dataset lang    cnt
  body_en   en 100000
  body_ja   ja 100000
titles_en   en 100000
titles_ja   ja 100000

インデックス:
index_name table_name  is_unique
  hnsw_idx  documents      False

Embedding次元: 1024


In [21]:
# 類似検索テスト
print('\n' + '=' * 80)
print('類似検索テスト')
print('=' * 80)

# 日本語本文からランダムなドキュメントをクエリとして使用
query_doc = con.execute("""
    SELECT id, title, embedding FROM documents WHERE dataset = 'body_ja' LIMIT 1
""").fetchone()

query_id, query_title, query_emb = query_doc
print(f'\nクエリ: id={query_id}, title="{query_title[:40]}..."')

result = con.execute("""
    SELECT id, title, lang, dataset,
           array_cosine_similarity(embedding, ?::FLOAT[1024]) as similarity
    FROM documents
    WHERE id != ?
    ORDER BY similarity DESC
    LIMIT 5
""", [list(query_emb), query_id]).fetchdf()

print('\nTop-5 類似結果:')
print(result.to_string(index=False))

con.close()
print('\n検証完了')


類似検索テスト

クエリ: id=300000, title="Category:成都のスポーツ競技大会..."

Top-5 類似結果:
    id                    title lang dataset  similarity
308723           Category:成都の歴史   ja body_ja    0.942124
316256           Category:成都の大学   ja body_ja    0.936409
318488 Category:ホーチミン市のスポーツ競技大会   ja body_ja    0.935011
352566         Category:成都市の鉄道駅   ja body_ja    0.932598
303197         Category:杭州のスポーツ   ja body_ja    0.930130

検証完了


<cell_type>markdown</cell_type>## 検証結果サマリー

### 基本情報

| 項目 | 値 |
|------|-----|
| ファイル | `data/experiment_400k.duckdb` |
| サイズ | 約6.8 GB |
| 総レコード数 | 400,000件 |

### データセット別件数

| データセット | 言語 | 件数 |
|-------------|------|------|
| titles_en | en | 100,000 |
| titles_ja | ja | 100,000 |
| body_en | en | 100,000 |
| body_ja | ja | 100,000 |

### カラム構造

| カラム | 型 | 説明 |
|--------|-----|------|
| id | INTEGER | 連番ID (0-399,999) |
| original_id | INTEGER | 元Wikipedia記事ID |
| title | VARCHAR | 記事タイトル |
| lang | VARCHAR | 言語 (en/ja) |
| dataset | VARCHAR | データセット名 |
| text | VARCHAR | 元テキスト |
| text_truncated | VARCHAR | 切り詰めテキスト (最大2000文字) |
| embedding | FLOAT[1024] | E5-large埋め込み |

### インデックス

- **HNSW** (`hnsw_idx`) - cosine距離

### 検証結果

- 類似検索が正常に動作
- 関連するドキュメントが上位に返される

---

**大規模実験の準備完了**

In [19]:
print('=' * 80)
print('実験用DuckDB作成完了')
print('=' * 80)
print(f'''
ファイル: {DB_PATH}
サイズ: {size_mb:.1f} MB

テーブル: documents
  - id: 連番ID (0-399,999)
  - original_id: 元のWikipedia記事ID
  - title: 記事タイトル
  - lang: 言語 (en/ja)
  - dataset: データセット名
  - text: 元テキスト（タイトル+本文）
  - text_truncated: 切り詰めテキスト（最大{MAX_CHARS}文字）
  - embedding: E5-large埋め込み (1024次元)

インデックス: HNSW (cosine)

データ内訳:
  - 英語タイトル: 100,000件
  - 日本語タイトル: 100,000件
  - 英語本文: 100,000件
  - 日本語本文: 100,000件
  - 合計: 400,000件
''')

実験用DuckDB作成完了

ファイル: ../data/experiment_400k.duckdb
サイズ: 6844.8 MB

テーブル: documents
  - id: 連番ID (0-399,999)
  - original_id: 元のWikipedia記事ID
  - title: 記事タイトル
  - lang: 言語 (en/ja)
  - dataset: データセット名
  - text: 元テキスト（タイトル+本文）
  - text_truncated: 切り詰めテキスト（最大2000文字）
  - embedding: E5-large埋め込み (1024次元)

インデックス: HNSW (cosine)

データ内訳:
  - 英語タイトル: 100,000件
  - 日本語タイトル: 100,000件
  - 英語本文: 100,000件
  - 日本語本文: 100,000件
  - 合計: 400,000件

